In [1]:
#import os
#import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

tf.get_logger().setLevel('ERROR')

In [2]:
RANDOM_STATE = 44
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32

In [5]:
tweets_df = pd.read_csv('../data/train.csv', index_col='id')

In [6]:
tweets_df.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
tweet_data = tweets_df.pop('text')
target = tweets_df.pop('target')

In [10]:
type(tweet_data)

pandas.core.series.Series

In [11]:
X_train, X_val, y_train, y_val = train_test_split(tweet_data, target, test_size=0.2, random_state=RANDOM_STATE)

In [12]:
raw_dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [15]:
i=0
for x,y in raw_dataset_train:
    print(type(x),x)
    print(type(y), y)
    i += 1
    if i==3:
        break

<class 'tensorflow.python.framework.ops.EagerTensor'> tf.Tensor(b'Japan Heat Wave Intensifies \xc2\x89\xc3\x9b\xc3\x92 Death Toll Surges To 55  http://t.co/irpSSresRq', shape=(), dtype=string)
<class 'tensorflow.python.framework.ops.EagerTensor'> tf.Tensor(1, shape=(), dtype=int64)
<class 'tensorflow.python.framework.ops.EagerTensor'> tf.Tensor(b'@Bang_Me_Up_Guk he was ;-; like he was singing so loud ;-;', shape=(), dtype=string)
<class 'tensorflow.python.framework.ops.EagerTensor'> tf.Tensor(0, shape=(), dtype=int64)
<class 'tensorflow.python.framework.ops.EagerTensor'> tf.Tensor(b'@MarianKeyes Rubber Mudslide! Still laughing!', shape=(), dtype=string)
<class 'tensorflow.python.framework.ops.EagerTensor'> tf.Tensor(0, shape=(), dtype=int64)


In [16]:
raw_dataset_val = tf.data.Dataset.from_tensor_slices((X_val, y_val))

In [14]:
for feat, targ in raw_dataset_train.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: b'Japan Heat Wave Intensifies \xc2\x89\xc3\x9b\xc3\x92 Death Toll Surges To 55  http://t.co/irpSSresRq', Target: 1
Features: b'@Bang_Me_Up_Guk he was ;-; like he was singing so loud ;-;', Target: 0
Features: b'@MarianKeyes Rubber Mudslide! Still laughing!', Target: 0
Features: b'Armageddon https://t.co/uCSUDk3q1d', Target: 1
Features: b'Beyond all bounds; till inundation rise', Target: 0


In [15]:
len(raw_dataset_train), len(raw_dataset_val)

(6090, 1523)

In [16]:
dataset_train = raw_dataset_train.shuffle(len(raw_dataset_train)).batch(batch_size=batch_size).cache().prefetch(buffer_size=AUTOTUNE)


In [17]:
dataset_val = raw_dataset_val.shuffle(len(raw_dataset_val)).batch(batch_size=batch_size).cache().prefetch(buffer_size=AUTOTUNE)

In [18]:
type(raw_dataset_train)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [31]:
for text_batch, label_batch in dataset_train.take(1):
    for i in range(3):
        print(f'Review: {text_batch.numpy()[i]}')
        label = label_batch.numpy()[i]
        print(f'Label : {label} ({label == 1})')

Review: b'@PyrBliss ah I remember those days. In a snowstorm too.'
Label : 1 (True)
Review: b'IS ROSS DEAD NOOOOOOOOOOOO @MikeParrActor'
Label : 0 (False)
Review: b'Truth...\nhttps://t.co/h6amECX5K7\n#News\n#BBC\n#CNN\n#Islam\n#Truth\n#god\n#ISIS\n#terrorism\n#Quran\n#Lies http://t.co/B8iWRdxcm0'
Label : 0 (False)


In [30]:
for x,y in dataset_train.take(2):
    print(type(x))
    print(type(y))
    print(x)
    print(y)

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[b'Photo: weallheartonedirection: I wouldn\xc2\x89\xc3\x9b\xc2\xaat let David electrocute himself so I\xc2\x89\xc3\x9b\xc2\xaam the asshole http://t.co/uWiJMEGl4E'
 b'Run out evacuation hospital indexing remedial of angioplasty dissertation at power elite hospitals dismayed:\xc2\x89\xc3\x9b_ http://t.co/VGvJGr8zoO'
 b'Hiroshima: They told me to paint my story: Eighty-nine year old man recalls the terror and the trauma when the... http://t.co/spE7U8t40K'
 b"Politics = Preschool Attitude: Russia orders to destroy all food coming from countries it doesn't like. - There is no hunger in the world?"
 b'A memorial to the millions who perished in the Holodomor has been erected in the U.S. capital. http://t.co/Dj1LWZNIEH http://t.co/I9MxXkzHbL'
 b"why wasn't this warship sunk?      CNN: First on CNN: Iranian warship points weapon at U.S. helicopter official says http://t.co/hfZ

In [32]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [33]:
bert_preprocess_model

In [35]:
text_test = ['OMG Horrible Accident Man Died in Wings of Airplane. http://t.co/xDxDPrcPnS. I am so scared.', 'this is not good']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_type_ids', 'input_word_ids']
Shape      : (2, 128)
Word Ids   : [  101 18168  2290  9202  4926  2158  2351  1999  4777  1997 13297  1012]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [37]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(2, 512)
Pooled Outputs Values:[ 0.9248217   0.99832433 -0.1196861   0.13600923 -0.36519977  0.93235236
  0.996665   -0.9902595   0.27941045 -0.99579823 -0.02256509 -0.99463373]
Sequence Outputs Shape:(2, 128, 512)
Sequence Outputs Values:[[ 0.8901094   0.53639644  0.44512567 ... -0.05968687 -0.42927226
   0.51187897]
 [ 0.8317982   0.7554441   0.96159834 ... -0.3270962   0.22416791
  -1.4370451 ]
 [-1.5691924  -0.22528432  0.40620172 ... -0.11041026 -0.44720194
  -0.4154801 ]
 ...
 [-0.5500458  -0.7653843   0.5824063  ... -0.9629426  -0.13816576
   0.31517535]
 [ 0.5901615  -0.1198364  -0.47520694 ...  0.25475895 -0.08762254
  -0.02219763]
 [ 0.8945195   0.67119944 -0.6091186  ...  0.0266239   0.11308935
   0.39829618]]


In [38]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [40]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor(
[[0.39887214]
 [0.6001508 ]], shape=(2, 1), dtype=float32)


In [41]:
tf.keras.utils.plot_model(classifier_model)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [42]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [43]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(dataset_train).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [45]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [46]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=dataset_train,
                               validation_data=dataset_val,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/5
  3/191 [..............................] - ETA: 1:46:24 - loss: 0.7735 - binary_accuracy: 0.5712

KeyboardInterrupt: 

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
dataset_name = 'tweet'
saved_model_path = './{}_bert'.format(dataset_name.replace('/', '_'))

classifier_model.save(saved_model_path, include_optimizer=False)

In [ ]:
saved_model_path

'./tweet_bert'

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

In [ ]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'this is such an amazing movie!',  # this is the same sentence tried earlier
    'The movie was great!',
    'The movie was meh.',
    'The movie was okish.',
    'The movie was terrible...'
]

reloaded_results = tf.sigmoid(reloaded_model(tf.constant(examples)))
original_results = tf.sigmoid(classifier_model(tf.constant(examples)))

print('Results from the saved model:')
print_my_examples(examples, reloaded_results)
print('Results from the model in memory:')
print_my_examples(examples, original_results)

Results from the saved model:
input: this is such an amazing movie! : score: 0.032112
input: The movie was great!           : score: 0.034636
input: The movie was meh.             : score: 0.053057
input: The movie was okish.           : score: 0.068383
input: The movie was terrible...      : score: 0.056575

Results from the model in memory:
input: this is such an amazing movie! : score: 0.032112
input: The movie was great!           : score: 0.034636
input: The movie was meh.             : score: 0.053057
input: The movie was okish.           : score: 0.068383
input: The movie was terrible...      : score: 0.056575



In [ ]:
serving_results = reloaded_model \
            .signatures['serving_default'](tf.constant(examples))

serving_results = tf.sigmoid(serving_results['classifier'])

print_my_examples(examples, serving_results)

input: this is such an amazing movie! : score: 0.032112
input: The movie was great!           : score: 0.034636
input: The movie was meh.             : score: 0.053057
input: The movie was okish.           : score: 0.068383
input: The movie was terrible...      : score: 0.056575

